In [1]:
import torch
import torch.nn.functional as F

In [2]:
print(torch.__version__)

2.2.2+cu121


In [5]:
names = open("names.txt" , "r" , encoding="utf8").read().splitlines()

In [9]:
chars = ['.'] + sorted(list(set("".join(names))))
stoi = {ch : i for i , ch in enumerate(chars)}
itos = {i : ch for i , ch in enumerate(chars)}

In [10]:
minLen = min(len(i) for i in names)
maxLen = max(len(i) for i in names)
minLen , maxLen

(2, 20)

In [11]:
BLOCK_SIZE = 8
N_EMBED = 10
HIDDEN = 200

In [12]:
def build_dataset(words):
    x , y = [] , []
    for w in words:
        context = [0] * BLOCK_SIZE
        for i in w + '.':
            ix = stoi[i]
            x.append(context)
            y.append(ix)
            context = context[1:] + [ix]
    return torch.tensor(x) , torch.tensor(y)

In [13]:
n1 = int(0.8 * len(names))
n2 = int(0.9 * len(names))

In [14]:
x_train , y_train = build_dataset(names[:n1])
x_val , y_val = build_dataset(names[n1 : n2])
x_test , y_test = build_dataset(names[n2:])

In [16]:
import torch.nn as nn

In [18]:
layers = [
    nn.Embedding(len(chars) , N_EMBED),
    nn.Flatten() , 
    nn.Linear(BLOCK_SIZE * N_EMBED , HIDDEN),
    nn.Tanh(),
    nn.Linear(HIDDEN , len(chars))
]

In [19]:
params = []
for layer in layers:
    params.extend(layer.parameters())

In [20]:
for p in params:
    p.requires_grad = True

In [30]:
EPOCHS = 50000
BATCH_SIZE = 256

In [31]:
for i in range(EPOCHS):
    ix = torch.randint(0 , x_train.shape[0] , (BATCH_SIZE , ))
    xtr = x_train[ix]
    ytr = y_train[ix]
    x = xtr
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x , ytr)
    for p in params:
        p.grad = None
    loss.backward()
    lr = 0.01 if i < 25000 else 0.001
    for p in params:
        p.data += -lr * p.grad
    if i % 10000 == 0:
        print(f"{i} / {EPOCHS} -> {loss:.4f}")

0 / 50000 -> 1.7274
10000 / 50000 -> 1.6724
20000 / 50000 -> 1.7146
30000 / 50000 -> 1.7861
40000 / 50000 -> 1.8459


In [38]:
with torch.no_grad():
    x = x_val
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x , y_val)
    print(f"Val Loss -> {loss.item():.4f}")

Val Loss -> 2.7601


In [39]:
with torch.no_grad():
    x = x_test
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x , y_test)
    print(f"Val Loss -> {loss.item():.4f}")

Val Loss -> 2.7879


In [45]:
with torch.no_grad():
    inp = input("Enter the first character : ")
    inp2 = input("Enter the second character : ")
    inp3 = input("Enter the third character : ")
    for i in range(10):
        out = [stoi[inp] , stoi[inp2] , stoi[inp3]]
        context = [0] * 5 + [stoi[inp] , stoi[inp2] , stoi[inp3]]
        length = 0
        while (length < 100):
            length += 1
            x = torch.tensor([context])
            for layer in layers:
                x = layer(x)
            probs = F.softmax(x , dim = 1)
            ix = torch.multinomial(probs , 1).item()
            context = context[1:] + [ix]
            if (ix == 0):
                break
            out.append(ix)
        print("".join(itos[i] for i in out))


namarev
nampa
namshov
namov
namb
namles
nambett
namobawa
namokov
nampa


In [ ]:
""